# 1. Load and handling data

- Doc inspire:
https://arxiv.org/pdf/2508.05633
- Data dictionary:
https://imgkkk574.github.io/KuaiLive/detailed_statistics.html

1. Load and handling data
2. EDA

2.1. Demographic: users vs. streamers, genders, age

2.2. Streamer activity analysis:

- Engagement distribution: numbers of user interactions
- Longtail Popularity distribution
- Time of streaming behaviors

2.3. User activity analysis:

- Engagement distribution
- Time of activity
- Repeat Consumption behavior
3. Assignments:

3.1. User Retention?

3.2. User LTV?

3.3. CTR KPIs: funnel pipeline?

## 1.1 Load data

In [1]:
# load lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# I want to show all columns in the dataframe
pd.set_option('display.max_columns', None)

In [23]:
# import dataset
click = pd.read_csv('click.csv')
comment = pd.read_csv('comment.csv')
gift = pd.read_csv('gift.csv')
like = pd.read_csv('like.csv')
negative = pd.read_csv('negative.csv')
room = pd.read_csv('room.csv')
streamer = pd.read_csv('streamer.csv')
user = pd.read_csv('user.csv')

In [16]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23772 entries, 0 to 23771
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   user_id                     23772 non-null  int64 
 1   age                         23772 non-null  object
 2   gender                      23772 non-null  object
 3   country                     23772 non-null  object
 4   device_brand                23772 non-null  object
 5   device_price                23772 non-null  object
 6   reg_timestamp               23772 non-null  object
 7   fans_num                    23772 non-null  object
 8   follow_num                  23772 non-null  object
 9   first_watch_live_timestamp  23772 non-null  object
 10  accu_watch_live_cnt         23772 non-null  object
 11  accu_watch_live_duration    23772 non-null  object
 12  is_live_streamer            23772 non-null  int64 
 13  is_photo_author             23772 non-null  in

In [24]:
# explore the datasets
room.head()

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id
0,20250525,7336601,252634,1,1748131180878,1748275200000,shop,0
1,20250525,10417334,329591,1,1748147435572,1748275200000,other,1
2,20250525,1396642,243911,2,1748175509619,1748275200000,other,2
3,20250525,5564711,21022,1,1748169384061,1748275200000,other,3
4,20250525,7236921,232913,2,1748169106971,1748275200000,other,4


In [ ]:
# my timestamp is in this format 1746374400819 (int datatype), pls convert it to datetime. I may have 1 or 2 timestamp columns in each dataset.
def convert_timestamp_to_datetime(df, *timestamp_columns):
    for col in timestamp_columns:
        if col in df.columns:
            # Only convert non-null values
            df[col] = pd.to_datetime(df[col], unit='ms')
            # Convert to China time
            df[col] = df[col] + pd.Timedelta(hours=8)
    return df

# now convert
click = convert_timestamp_to_datetime(click, 'timestamp')
comment = convert_timestamp_to_datetime(comment, 'timestamp')
gift = convert_timestamp_to_datetime(gift, 'timestamp')
like = convert_timestamp_to_datetime(like, 'timestamp')
negative = convert_timestamp_to_datetime(negative, 'timestamp')
room = convert_timestamp_to_datetime(room, 'start_timestamp', 'end_timestamp')

# now test again
room.head()

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id
0,20250525,7336601,252634,1,2025-05-25 07:59:40.878,2025-05-27,shop,0
1,20250525,10417334,329591,1,2025-05-25 12:30:35.572,2025-05-27,other,1
2,20250525,1396642,243911,2,2025-05-25 20:18:29.619,2025-05-27,other,2
3,20250525,5564711,21022,1,2025-05-25 18:36:24.061,2025-05-27,other,3
4,20250525,7236921,232913,2,2025-05-25 18:31:46.971,2025-05-27,other,4


- Room means that in a live session, users are invited into a room with that streamer!
- Interesting to note, some rooms have end_timestamp as 2025-05-26 16:00:00 --> it is the decode that those rooms are still online up to the data cutoff date. We understand that the data is consist of 21-day collection window.
- Remember that we loaded into the UTC time, which is 8 hours behind vs. China. So we need to create a def to adjust the time back to China time

In [27]:
click.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time
0,8505,9342705,392199,2025-05-05 00:00:00.022,2852
1,8505,9962836,412107,2025-05-05 00:00:00.022,1906
2,8505,3286104,243272,2025-05-05 00:00:00.022,7356
3,22375,7464709,18453,2025-05-05 00:00:00.362,94895
4,23154,7865151,433825,2025-05-05 00:00:00.819,94474


In [28]:
# convert watch_live_time (in ms) to minute
click['watch_live_time'] = click['watch_live_time'] / 1000 / 60
click.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time
0,8505,9342705,392199,2025-05-05 00:00:00.022,0.047533
1,8505,9962836,412107,2025-05-05 00:00:00.022,0.031767
2,8505,3286104,243272,2025-05-05 00:00:00.022,0.122600
3,22375,7464709,18453,2025-05-05 00:00:00.362,1.581583
4,23154,7865151,433825,2025-05-05 00:00:00.819,1.574567


In [29]:
click.head(50)

,user_id,live_id,streamer_id,timestamp,watch_live_time
0,8505,9342705,392199,2025-05-05 00:00:00.022,0.047533
1,8505,9962836,412107,2025-05-05 00:00:00.022,0.031767
2,8505,3286104,243272,2025-05-05 00:00:00.022,0.122600
3,22375,7464709,18453,2025-05-05 00:00:00.362,1.581583
4,23154,7865151,433825,2025-05-05 00:00:00.819,1.574567
5,19622,9814921,106495,2025-05-05 00:00:01.145,0.043117
6,19622,6689260,309584,2025-05-05 00:00:01.145,0.020083
7,19622,10999279,100157,2025-05-05 00:00:01.145,0.046700
8,19622,2013427,211160,2025-05-05 00:00:01.145,0.038783
9,19622,2780253,196575,2025-05-05 00:00:01.145,0.020117


In [31]:
comment.head()

,user_id,live_id,streamer_id,timestamp
0,23154,7865151,433825,2025-05-05 00:00:00.819
1,21092,310545,85826,2025-05-05 00:00:02.301
2,5222,541927,244121,2025-05-05 00:00:14.059
3,3980,10638726,332836,2025-05-05 00:00:27.355
4,6939,1238748,225831,2025-05-05 00:00:28.171


In [32]:
gift.head()

,user_id,live_id,streamer_id,timestamp,gift_price
0,11504,6086847,114419,2025-05-05 00:00:41.260,2
1,7742,3421616,31236,2025-05-05 00:01:01.781,6
2,12541,7372555,360222,2025-05-05 00:01:13.533,1
3,896,6783244,17845,2025-05-05 00:01:34.010,12
4,19945,10149732,231687,2025-05-05 00:02:08.713,2


In [34]:
like.head()

,user_id,live_id,streamer_id,timestamp
0,5222,541927,244121,2025-05-05 00:00:14.059
1,2178,5438872,291618,2025-05-05 00:00:18.706
2,1421,1112877,403243,2025-05-05 00:00:26.541
3,3980,10638726,332836,2025-05-05 00:00:27.355
4,10666,2840350,162352,2025-05-05 00:00:27.803


In [35]:
negative.head()

,user_id,live_id,streamer_id,timestamp
0,9810,10816308,17452,2025-05-06 18:05:26.498
1,13111,2681806,7538,2025-05-06 18:06:35.894
2,18341,8166810,140041,2025-05-06 18:09:12.814
3,18341,4736019,276818,2025-05-06 18:09:12.814
4,10014,7337508,237678,2025-05-06 18:09:35.990


The file negative.csv contains records of all exposures that were presented to users but not clicked. Each record includes the corresponding user, live room, streamer, and the timestamp of the exposure.

In [36]:
streamer.head()

,streamer_id,gender,age,country,device_brand,device_price,live_operation_tag,fans_user_num,fans_group_fans_num,follow_user_num,first_live_timestamp,accu_live_cnt,accu_live_duration,accu_play_cnt,accu_play_duration,reg_timestamp,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6
0,56006,M,24-30,China,APPLE,5000+,Relationship,10000-100000,0-10,1000-10000,2018-02-04,100-500,500000000-1000000000,100000-500000,50000000000-100000000000,2014-07-03,1,0,0,0,0,0,0
1,354478,M,24-30,China,APPLE,5000+,Chat,100-1000,10-50,10-100,2016-07-31,100-500,1000000000-3000000000,0-50000,0-1000000000,2014-09-11,1,0,0,0,0,0,0
2,357970,M,24-30,China,APPLE,5000+,Other,100-1000,0-10,100-1000,2024-12-26,0-50,0-100000000,0-50000,0-1000000000,2014-09-17,1,0,0,0,0,0,0
3,108285,F,18-23,China,APPLE,5000+,Beauty,10000-100000,500-1000,100-1000,2018-02-19,5000-10000,10000000000-50000000000,1000000-5000000,100000000000-500000000000,2014-09-28,1,1,0,0,0,0,0
4,36951,M,31-40,China,APPLE,5000+,Chat,100000-1000000,5000-10000,1000-10000,2016-10-21,1000-5000,10000000000-50000000000,10000000+,1000000000000-5000000000000,2014-10-12,0,0,0,0,0,0,0


In [37]:
user.head()

,user_id,age,gender,country,device_brand,device_price,reg_timestamp,fans_num,follow_num,first_watch_live_timestamp,accu_watch_live_cnt,accu_watch_live_duration,is_live_streamer,is_photo_author,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6
0,22733,18-23,M,China,DESKTOP,0,2023-05-03,0-10,10-100,2023-05-03,0-50000,0-1000000000,0,0,0,0,0,0,0,0,0
1,3085,50+,M,China,HONOR,2000-4000,2023-07-11,10-100,100-1000,2023-07-12,0-50000,1000000000-5000000000,0,1,1,1,0,0,0,0,0
2,10604,41-49,M,China,HUAWEI,2000-4000,2017-02-14,100-1000,10-100,2017-10-04,0-50000,1000000000-5000000000,1,1,1,0,0,0,0,0,0
3,7056,31-40,M,China,OPPO,1000-2000,2017-09-06,10-100,10-100,2017-09-06,0-50000,0-1000000000,0,1,1,0,0,1,0,0,0
4,3791,41-49,M,China,HONOR,1000-2000,2019-05-21,1000-10000,1000-10000,2019-05-21,100000-500000,10000000000-50000000000,1,1,0,0,0,0,0,0,0


- User features have:

gender, age, country, device_brand, device_price, reg_timestamp, fans_num, follow_num,
first_watch_live_timestamp, accu_watch_live_cnt, accu_watch_live_duration, is_live_author, is_video_author,
and 7 encrypted vectors.
- Streamer features have:

gender, age, country, device_brand, device_price, reg_timestamp, live_operation_tag, fans_num, fans_group_num,
follow_num, first_live_timestamp, accu_live_cnt, accu_live_duration, accu_play_cnt, accu_play_duration,
and 7 encrypted vectors

## 1.2 Data cleaning

### Remove odd live_ids

OK, now lets remove out odds live_id that's have more than 1 day live, and live_id has no ending

In [45]:
# let's calculate live duration for each room
room['live_dur_day'] = room['end_timestamp'] - room['start_timestamp']
room['live_dur'] = room['live_dur_day'].dt.total_seconds() / 60
room.head()

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id,live_dur,live_dur_day
0,20250525,7336601,252634,1,2025-05-25 07:59:40.878,2025-05-27,shop,0,2400.318700,1 days 16:00:19.122000
1,20250525,10417334,329591,1,2025-05-25 12:30:35.572,2025-05-27,other,1,2129.407133,1 days 11:29:24.428000
2,20250525,1396642,243911,2,2025-05-25 20:18:29.619,2025-05-27,other,2,1661.506350,1 days 03:41:30.381000
3,20250525,5564711,21022,1,2025-05-25 18:36:24.061,2025-05-27,other,3,1763.598983,1 days 05:23:35.939000
4,20250525,7236921,232913,2,2025-05-25 18:31:46.971,2025-05-27,other,4,1768.217150,1 days 05:28:13.029000


In [44]:
room.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11819964 entries, 0 to 11819963
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   p_date                 int64         
 1   live_id                int64         
 2   streamer_id            int64         
 3   live_type              int64         
 4   start_timestamp        datetime64[ns]
 5   end_timestamp          datetime64[ns]
 6   live_content_category  object        
 7   live_name_id           int64         
 8   live_dur               float64       
dtypes: datetime64[ns](2), float64(1), int64(5), object(1)
memory usage: 811.6+ MB


I see some livestreams are more than 1 day, is that make any senses?

In [46]:
room.sort_values(by='live_dur', ascending=False).head(10)

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id,live_dur,live_dur_day
6394866,20250516,10992456,400652,-124,2025-05-09 10:59:51.566,2025-05-16 11:00:10.249,shop,50,10080.311383,7 days 00:00:18.683000
10628178,20250523,10914327,180176,-124,2025-05-16 21:40:09.082,2025-05-23 21:40:27.729,other,213262,10080.310783,7 days 00:00:18.647000
11079833,20250524,8013892,107086,-124,2025-05-17 18:46:09.854,2025-05-24 18:46:28.270,other,24,10080.306933,7 days 00:00:18.416000
8685791,20250520,3687857,322604,1,2025-05-13 12:07:07.545,2025-05-20 12:07:24.727,shop,43797,10080.286367,7 days 00:00:17.182000
732755,20250506,4330706,303924,-124,2025-04-29 12:51:40.680,2025-05-06 12:51:54.973,other,189862,10080.238217,7 days 00:00:14.293000
7732575,20250518,2908547,394987,-124,2025-05-11 19:26:45.749,2025-05-18 19:26:58.997,other,24,10080.220800,7 days 00:00:13.248000
946263,20250506,3322554,192644,-124,2025-04-29 20:53:08.757,2025-05-06 20:53:21.046,other,24,10080.204817,7 days 00:00:12.289000
5027553,20250513,5328381,33412,1,2025-05-06 23:06:02.108,2025-05-13 23:06:11.261,shop,223017,10080.152550,7 days 00:00:09.153000
10622124,20250523,7927291,192644,-124,2025-05-16 21:31:56.953,2025-05-23 21:32:04.515,other,24,10080.126033,7 days 00:00:07.562000
885836,20250506,1901816,321370,-124,2025-04-29 18:50:44.215,2025-05-06 18:50:50.163,other,33,10080.099133,7 days 00:00:05.948000


In [48]:
# find out the room with live_dur greater than 1 day
room[room['live_dur_day'] > pd.Timedelta(days=1)].shape

(14613, 10)

In [49]:
room.shape

(11819964, 10)

- Around 0.1% live_id that has more than 1 day --> suggest to remove them all of datasets
- Let's take a list of live_id that have problems

In [50]:
problematic_live_ids = room['live_id'][room['live_dur_day'] > pd.Timedelta(days=1)].unique()

In [51]:
problematic_live_ids.shape[0], room['live_id'].shape[0], problematic_live_ids.shape[0] / room['live_id'].shape[0] * 100

(13849, 11819964, 0.11716617749428002)

In [52]:
# create a def to remove these live_id from all datasets
def remove_live_id(df, problematic_live_ids):
    return df[~df['live_id'].isin(problematic_live_ids)]

# remove problematic live_id from all datasets
click_cl = remove_live_id(click, problematic_live_ids)
comment_cl = remove_live_id(comment, problematic_live_ids)
gift_cl = remove_live_id(gift, problematic_live_ids)
like_cl = remove_live_id(like, problematic_live_ids)
negative_cl = remove_live_id(negative, problematic_live_ids)
room_cl = remove_live_id(room, problematic_live_ids)

# check 1 shape of room_cl
room_cl.shape

(11805351, 10)

In [53]:
click_cl.shape, click.shape, click_cl.shape[0] / click.shape[0] * 100 # Only 0.4% data removed, so its safe now!

((4888612, 5), (4909515, 5), 99.5742349295195)

In [54]:
comment_cl.shape, comment.shape, comment_cl.shape[0] / comment.shape[0] * 100 # Only 0.1% data removed, so its safe now!

((196020, 4), (196526, 4), 99.7425277062577)

### Should we remove odd user_id that watch over 1 day?

In [58]:
click_cl.sort_values(by='watch_live_time', ascending=False)

,user_id,live_id,streamer_id,timestamp,watch_live_time
1340052,928,9238724,53517,2025-05-09 23:30:35.430,119.594800
853556,3998,10648889,146570,2025-05-08 07:52:49.461,119.297800
4876561,20511,1327008,315160,2025-05-25 20:07:34.735,118.581567
2569770,10205,11412521,335970,2025-05-14 23:00:00.590,118.561433
3456227,3103,8464071,320106,2025-05-18 19:05:45.985,118.425133
...,...,...,...,...,...
516743,18154,310736,159095,2025-05-06 23:23:36.490,0.000100
1228174,5041,10238216,241282,2025-05-09 15:06:45.485,0.000083
1954043,23033,894226,187603,2025-05-12 16:07:54.373,0.000083
3792823,363,4298869,313499,2025-05-20 02:10:18.054,0.000083


There is no problem with this. Heavy user watched for 120 mins = 2 hours, its fine!

### Is live_id unique?

In [61]:
room_cl.sort_values(by='live_id', ascending=False).head(10)

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id,live_dur,live_dur_day
10345384,20250523,11613708,306089,1,2025-05-23 10:43:08.303,2025-05-23 11:07:45.031,other,1432,24.612133,0 days 00:24:36.728000
9464976,20250521,11613707,108409,1,2025-05-21 19:03:41.727,2025-05-21 20:01:28.742,other,354319,57.783583,0 days 00:57:47.015000
3932651,20250512,11613706,370166,1,2025-05-12 00:20:30.332,2025-05-12 00:21:52.691,other,-1,1.372650,0 days 00:01:22.359000
10691886,20250523,11613705,397675,1,2025-05-23 20:08:20.393,2025-05-23 23:03:52.053,other,1435,175.527667,0 days 02:55:31.660000
10691887,20250523,11613705,397675,1,2025-05-23 20:08:20.393,2025-05-23 23:03:52.053,other,551694,175.527667,0 days 02:55:31.660000
5126220,20250514,11613704,302106,1,2025-05-13 17:35:10.875,2025-05-14 03:34:13.590,other,40280,599.045250,0 days 09:59:02.715000
7484044,20250518,11613703,144642,1,2025-05-18 08:56:50.361,2025-05-18 09:10:35.384,other,63261,13.750383,0 days 00:13:45.023000
7555167,20250518,11613702,41043,1,2025-05-18 11:00:19.372,2025-05-18 12:15:08.028,other,284863,74.810933,0 days 01:14:48.656000
3911414,20250511,11613701,273858,1,2025-05-11 20:36:57.662,2025-05-11 23:46:27.872,other,23103,189.503500,0 days 03:09:30.210000
9512399,20250521,11613700,316765,1,2025-05-21 21:11:15.200,2025-05-21 21:21:34.897,other,594021,10.328283,0 days 00:10:19.697000


In [62]:
room_cl.shape

(11805351, 10)

In [63]:
room_cl['live_id'].nunique()

11599859

- We see that live_id 11613705 is duplicate, all info is the same, just the live_name_id is different
- According to data dictionary, live_name_id is the id associated with the live room title, used to index the encoded title embeddings.
- Because we don't use embeddings, and live_name_id do not exist in any other df, so suggest to drop that column to make sure the room_cl has PK = live_id

In [64]:
# drop live_name_id column and take distinct rows
room_cl = room_cl.drop(columns=['live_name_id']).drop_duplicates(subset='live_id')
room_cl.shape

(11599859, 9)

In [65]:
room.sort_values(by=['live_id', 'streamer_id'], ascending=False).head(50)

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_name_id,live_dur,live_dur_day
10345384,20250523,11613708,306089,1,2025-05-23 10:43:08.303,2025-05-23 11:07:45.031,other,1432,24.612133,0 days 00:24:36.728000
9464976,20250521,11613707,108409,1,2025-05-21 19:03:41.727,2025-05-21 20:01:28.742,other,354319,57.783583,0 days 00:57:47.015000
3932651,20250512,11613706,370166,1,2025-05-12 00:20:30.332,2025-05-12 00:21:52.691,other,-1,1.372650,0 days 00:01:22.359000
10691886,20250523,11613705,397675,1,2025-05-23 20:08:20.393,2025-05-23 23:03:52.053,other,1435,175.527667,0 days 02:55:31.660000
10691887,20250523,11613705,397675,1,2025-05-23 20:08:20.393,2025-05-23 23:03:52.053,other,551694,175.527667,0 days 02:55:31.660000
5126220,20250514,11613704,302106,1,2025-05-13 17:35:10.875,2025-05-14 03:34:13.590,other,40280,599.045250,0 days 09:59:02.715000
7484044,20250518,11613703,144642,1,2025-05-18 08:56:50.361,2025-05-18 09:10:35.384,other,63261,13.750383,0 days 00:13:45.023000
7555167,20250518,11613702,41043,1,2025-05-18 11:00:19.372,2025-05-18 12:15:08.028,other,284863,74.810933,0 days 01:14:48.656000
3911414,20250511,11613701,273858,1,2025-05-11 20:36:57.662,2025-05-11 23:46:27.872,other,23103,189.503500,0 days 03:09:30.210000
9512399,20250521,11613700,316765,1,2025-05-21 21:11:15.200,2025-05-21 21:21:34.897,other,594021,10.328283,0 days 00:10:19.697000


Now it is correct!

## 1.3 Data Model

### Check unique values to find out PK

#### Dim tables

In [66]:
streamer['streamer_id'].nunique(), streamer.shape # Checked!

(452621, (452621, 23))

In [67]:
user['user_id'].nunique(), user.shape # Checked!

(23772, (23772, 21))

#### Fact tables

Check comment

In [68]:
comment_cl.head()

,user_id,live_id,streamer_id,timestamp
0,23154,7865151,433825,2025-05-05 00:00:00.819
1,21092,310545,85826,2025-05-05 00:00:02.301
2,5222,541927,244121,2025-05-05 00:00:14.059
3,3980,10638726,332836,2025-05-05 00:00:27.355
4,6939,1238748,225831,2025-05-05 00:00:28.171


In [69]:
# for comment_cl dataset, we may have a composit key of live_id and user_id, let's confirm it
comment_cl_test = comment_cl.copy()
comment_cl_test['composite_key'] = comment_cl_test['live_id'].astype(str) + '_' + comment_cl_test['user_id'].astype(str) + '_' + comment_cl_test['streamer_id'].astype(str)
comment_cl_test['composite_key'].nunique(), comment_cl_test.shape
# Not matching!

(185939, (196020, 5))

In [70]:
comment_cl_test.sort_values(by=['user_id', 'live_id', 'streamer_id'], ascending=False).head(50)

,user_id,live_id,streamer_id,timestamp,composite_key
196207,23772,11045516,309926,2025-05-25 22:16:23.179,11045516_23772_309926
155351,23772,10272210,161079,2025-05-20 23:19:01.303,10272210_23772_161079
184374,23772,8639240,365822,2025-05-24 15:33:24.890,8639240_23772_365822
193463,23772,8413004,447744,2025-05-25 17:04:02.248,8413004_23772_447744
174841,23772,7230833,361301,2025-05-23 13:53:49.204,7230833_23772_361301
133684,23772,6978298,4755,2025-05-18 19:46:28.845,6978298_23772_4755
193521,23772,6392661,278793,2025-05-25 17:11:24.304,6392661_23772_278793
86561,23772,6387257,161079,2025-05-13 19:31:34.465,6387257_23772_161079
184596,23772,6383837,279547,2025-05-24 16:13:07.114,6383837_23772_279547
193402,23772,6036775,133751,2025-05-25 16:55:22.985,6036775_23772_133751


I understand, so each rows is the comment record. 1 user may have more than 1 comment on 1 live_id. That's why, we need to define composite_key as live_user_1, live_user_2 etc. regarding to the comment number 1, 2, 3, etc. ranking by timestamp

In [71]:
def create_composite_key(df):
    """
    Create a composite key based on user_id, live_id, and rank.
    """
    # Sort by user_id, live_id, and timestamp to ensure correct ranking

    df = df.sort_values(['user_id', 'live_id', 'timestamp'])

    # Create the rank for each comment per user_id and live_id

    df['comment_rank'] = df.groupby(['user_id', 'live_id']).cumcount() + 1

    # Create the composite key

    df['composite_key'] = (
        df['user_id'].astype(str) + '_' +
        df['live_id'].astype(str) + '_' +
        df['comment_rank'].astype(str)
    )
    return df

In [72]:
# Apply
comment_cl_test = create_composite_key(comment_cl_test)
comment_cl_test.head()

,user_id,live_id,streamer_id,timestamp,composite_key,comment_rank
59514,1,2820681,116940,2025-05-10 22:26:23.181,1_2820681_1,1
117359,1,4489049,116940,2025-05-16 22:38:14.644,1_4489049_1,1
99121,2,3961718,128672,2025-05-14 22:37:34.301,2_3961718_1,1
183878,3,4330227,60834,2025-05-24 14:08:33.226,3_4330227_1,1
25417,3,5984199,7538,2025-05-07 17:41:00.318,3_5984199_1,1


In [73]:
# OK, not lets make it into comment_cl
comment_cl = comment_cl_test.copy()

Do the same for gift, like, negative

In [74]:
gift_cl_test = create_composite_key(gift_cl)
gift_cl_test.shape, gift_cl.shape

((72438, 7), (72438, 5))

In [75]:
like_cl_test = create_composite_key(like_cl)
like_cl_test.shape, like_cl.shape

((178803, 6), (178803, 4))

In [76]:
click_cl_test = create_composite_key(click_cl)
click_cl_test.shape, click_cl.shape

((4888612, 7), (4888612, 5))

In [77]:
negative_cl_test = create_composite_key(negative_cl)
negative_cl_test.shape, negative_cl.shape

((12667907, 6), (12667907, 4))

In [78]:
# All good, lets assign these to the original dataframes
gift_cl = gift_cl_test.copy()
like_cl = like_cl_test.copy()
click_cl = click_cl_test.copy()
negative_cl = negative_cl_test.copy()

In [79]:
# Though user and streamer, we do not clean anything, but lets take _cl for consistency
user_cl = user.copy()
streamer_cl = streamer.copy()

In [80]:
# Let's create a p_date column in YYYYMMDD integer format for each dataframe
def create_p_date(df, date_col):
    """
    Create a p_date column in YYYYMMDD integer format based on the date_col.
    """
    df['p_date'] = df[date_col].dt.strftime('%Y%m%d').astype(int)
    return df

click_cl = create_p_date(click_cl, 'timestamp')
comment_cl = create_p_date(comment_cl, 'timestamp')
gift_cl = create_p_date(gift_cl, 'timestamp')
negative_cl = create_p_date(negative_cl, 'timestamp')
like_cl = create_p_date(like_cl, 'timestamp')

# view results
click_cl.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,comment_rank,composite_key,p_date
1304010,1,733942,192089,2025-05-09 20:27:14.571,0.146017,1,1_733942_1,20250509
1560773,1,1200655,248056,2025-05-10 22:50:26.713,0.312350,1,1_1200655_1,20250510
5125,1,1520913,285858,2025-05-05 00:25:02.111,0.075317,1,1_1520913_1,20250505
1556405,1,2820681,116940,2025-05-10 22:26:23.181,2.718033,1,1_2820681_1,20250510
1560772,1,2820681,116940,2025-05-10 22:50:26.713,0.245733,2,1_2820681_2,20250510


### Data Model

![demo-db (example).png](<attachment:demo-db (example).png>)

## 1.4 Create a big fact table

Let's have some info for dim tables

In [81]:
streamer_cl.head()

,streamer_id,gender,age,country,device_brand,device_price,live_operation_tag,fans_user_num,fans_group_fans_num,follow_user_num,first_live_timestamp,accu_live_cnt,accu_live_duration,accu_play_cnt,accu_play_duration,reg_timestamp,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6
0,56006,M,24-30,China,APPLE,5000+,Relationship,10000-100000,0-10,1000-10000,2018-02-04,100-500,500000000-1000000000,100000-500000,50000000000-100000000000,2014-07-03,1,0,0,0,0,0,0
1,354478,M,24-30,China,APPLE,5000+,Chat,100-1000,10-50,10-100,2016-07-31,100-500,1000000000-3000000000,0-50000,0-1000000000,2014-09-11,1,0,0,0,0,0,0
2,357970,M,24-30,China,APPLE,5000+,Other,100-1000,0-10,100-1000,2024-12-26,0-50,0-100000000,0-50000,0-1000000000,2014-09-17,1,0,0,0,0,0,0
3,108285,F,18-23,China,APPLE,5000+,Beauty,10000-100000,500-1000,100-1000,2018-02-19,5000-10000,10000000000-50000000000,1000000-5000000,100000000000-500000000000,2014-09-28,1,1,0,0,0,0,0
4,36951,M,31-40,China,APPLE,5000+,Chat,100000-1000000,5000-10000,1000-10000,2016-10-21,1000-5000,10000000000-50000000000,10000000+,1000000000000-5000000000000,2014-10-12,0,0,0,0,0,0,0


In [82]:
user_cl.head()

,user_id,age,gender,country,device_brand,device_price,reg_timestamp,fans_num,follow_num,first_watch_live_timestamp,accu_watch_live_cnt,accu_watch_live_duration,is_live_streamer,is_photo_author,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6
0,22733,18-23,M,China,DESKTOP,0,2023-05-03,0-10,10-100,2023-05-03,0-50000,0-1000000000,0,0,0,0,0,0,0,0,0
1,3085,50+,M,China,HONOR,2000-4000,2023-07-11,10-100,100-1000,2023-07-12,0-50000,1000000000-5000000000,0,1,1,1,0,0,0,0,0
2,10604,41-49,M,China,HUAWEI,2000-4000,2017-02-14,100-1000,10-100,2017-10-04,0-50000,1000000000-5000000000,1,1,1,0,0,0,0,0,0
3,7056,31-40,M,China,OPPO,1000-2000,2017-09-06,10-100,10-100,2017-09-06,0-50000,0-1000000000,0,1,1,0,0,1,0,0,0
4,3791,41-49,M,China,HONOR,1000-2000,2019-05-21,1000-10000,1000-10000,2019-05-21,100000-500000,10000000000-50000000000,1,1,0,0,0,0,0,0,0


For dim, we may want to have some info:
- Tenure: let's take cutoff day as 2025-05-26, and calculate tenure for both streamers and users since their first registration
- Active_time: also take a the active_time, means from the first watch_live (for users) or the first_live_time (for streamers) until 2025-05-26

In [83]:
# tenure
cutoff_day = pd.to_datetime('2025-05-26')
user_cl['tenure'] = (cutoff_day - pd.to_datetime(user_cl['reg_timestamp'])).dt.days
streamer_cl['tenure'] = (cutoff_day - pd.to_datetime(streamer_cl['reg_timestamp'])).dt.days

# active_time
user_cl['active_time'] = (cutoff_day - pd.to_datetime(user_cl['first_watch_live_timestamp'])).dt.days
streamer_cl['active_time'] = (cutoff_day - pd.to_datetime(streamer_cl['first_live_timestamp'])).dt.days

In [84]:
room_cl.head()

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_dur,live_dur_day
89,20250505,4207342,439076,1,2025-05-04 23:08:35.691,2025-05-05 00:00:00.035,other,51.405733,0 days 00:51:24.344000
90,20250505,6894755,356465,1,2025-05-04 22:35:55.074,2025-05-05 00:00:00.051,other,84.082950,0 days 01:24:04.977000
91,20250505,8396644,411360,1,2025-05-04 23:42:30.419,2025-05-05 00:00:00.093,other,17.494567,0 days 00:17:29.674000
92,20250505,4473195,271581,1,2025-05-04 22:35:47.484,2025-05-05 00:00:00.157,other,84.211217,0 days 01:24:12.673000
93,20250505,6447876,446221,1,2025-05-04 23:26:37.516,2025-05-05 00:00:00.253,other,33.378950,0 days 00:33:22.737000


Let note these fact tables name so we can concat them together 

In [85]:
click_cl['interaction_type'] = 'click'
comment_cl['interaction_type'] = 'comment'
gift_cl['interaction_type'] = 'gift'
like_cl['interaction_type'] = 'like'
negative_cl['interaction_type'] = 'negative'

Interactions overtime:
- We have click, comment, gift, like are interactions
- Negative means user skip / exit, we can treat as exit time
- live_id + user_id means in 1 session of that user interact with the live. We can create this column

In [86]:
# Concat all interactions into one dataframe (even negative)
interactions = pd.concat([click_cl, comment_cl, gift_cl, like_cl, negative_cl], ignore_index=True)
interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,comment_rank,composite_key,p_date,interaction_type,gift_price
0,1,733942,192089,2025-05-09 20:27:14.571,0.146017,1,1_733942_1,20250509,click,NaN
1,1,1200655,248056,2025-05-10 22:50:26.713,0.312350,1,1_1200655_1,20250510,click,NaN
2,1,1520913,285858,2025-05-05 00:25:02.111,0.075317,1,1_1520913_1,20250505,click,NaN
3,1,2820681,116940,2025-05-10 22:26:23.181,2.718033,1,1_2820681_1,20250510,click,NaN
4,1,2820681,116940,2025-05-10 22:50:26.713,0.245733,2,1_2820681_2,20250510,click,NaN


In [87]:
interactions.shape

(18003780, 10)

In [88]:
# Let's remove the comment_rank and composite_key. When map all data, it is not correct anymore
interactions.drop(columns=['comment_rank', 'composite_key'], inplace=True, errors='ignore')

# After concat, we can rank the interactions by user_id, live_id and timestamp, to see the order of interactions
interactions.sort_values(by=['user_id', 'live_id', 'timestamp'], inplace=True)
interactions['interaction_order'] = interactions.groupby(['user_id', 'live_id'])['timestamp'].rank(method='first')
interactions['interaction_order'] = interactions['interaction_order'].astype(int)

# Create a PK by combining user_id, live_id, and interaction_order
interactions['pk'] = interactions['user_id'].astype(str) + '_' + interactions['live_id'].astype(str) + '_' + interactions['interaction_order'].astype(str)

# Then create a new column to identify the session of the user in the live stream
interactions['session_id'] = interactions['user_id'].astype(str) + '_' + interactions['live_id'].astype(str)

# Result
interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,p_date,interaction_type,gift_price,interaction_order,pk,session_id
5335873,1,24005,307482,2025-05-10 12:46:21.212,NaN,20250510,negative,NaN,1,1_24005_1,1_24005
5335874,1,24005,307482,2025-05-10 13:42:39.811,NaN,20250510,negative,NaN,2,1_24005_2,1_24005
5335875,1,252326,169041,2025-05-09 23:05:42.101,NaN,20250509,negative,NaN,1,1_252326_1,1_252326
5335876,1,264714,193781,2025-05-11 00:37:06.659,NaN,20250511,negative,NaN,1,1_264714_1,1_264714
5335877,1,266768,430309,2025-05-05 12:25:33.964,NaN,20250505,negative,NaN,1,1_266768_1,1_266768


Lets map info from streamers & users, room as well
- Users, take their age, gender, tenure, active_time, accu_watch_live_cnt
- Streamers, take gender, age, live_operation_tag, fans_user_num
- Room, take the live_dur and live_content_category

In [89]:
# map interactions to user and take age, gender, active_time, accu_watch_live_cnt
interactions = interactions.merge(user_cl[['user_id', 'age', 'gender', 'active_time', 'accu_watch_live_cnt']], on='user_id', how='left')
interactions.rename(columns={'accu_watch_live_cnt': 'user_accu_watch_live_cnt', 'age' : 'user_age'
                             , 'gender' : 'user_gender', 'active_time' : 'user_active_time'}, inplace=True)

interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,p_date,interaction_type,gift_price,interaction_order,pk,session_id,user_age,user_gender,user_active_time,user_accu_watch_live_cnt
0,1,24005,307482,2025-05-10 12:46:21.212,NaN,20250510,negative,NaN,1,1_24005_1,1_24005,24-30,F,893,50000-100000
1,1,24005,307482,2025-05-10 13:42:39.811,NaN,20250510,negative,NaN,2,1_24005_2,1_24005,24-30,F,893,50000-100000
2,1,252326,169041,2025-05-09 23:05:42.101,NaN,20250509,negative,NaN,1,1_252326_1,1_252326,24-30,F,893,50000-100000
3,1,264714,193781,2025-05-11 00:37:06.659,NaN,20250511,negative,NaN,1,1_264714_1,1_264714,24-30,F,893,50000-100000
4,1,266768,430309,2025-05-05 12:25:33.964,NaN,20250505,negative,NaN,1,1_266768_1,1_266768,24-30,F,893,50000-100000


In [90]:
# map interactions to streamer and take age, gender, live_operation_tag, active_time, accu_live_cnt, fans_user_num
interactions = interactions.merge(streamer_cl[['streamer_id', 'age', 'gender', 'live_operation_tag', 'active_time', 'accu_live_cnt', 'fans_user_num']], on='streamer_id', how='left')
interactions.rename(columns={'live_operation_tag': 'streamer_live_operation_tag', 'age' : 'streamer_age'
                             , 'gender' : 'streamer_gender', 'active_time' : 'streamer_active_time'
                             ,'fans_user_num' : 'streamer_fans_user_num', 'accu_live_cnt' : 'streamer_accu_live_cnt'}, inplace=True)

interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,p_date,interaction_type,gift_price,interaction_order,pk,session_id,user_age,user_gender,user_active_time,user_accu_watch_live_cnt,streamer_age,streamer_gender,streamer_live_operation_tag,streamer_active_time,streamer_accu_live_cnt,streamer_fans_user_num
0,1,24005,307482,2025-05-10 12:46:21.212,NaN,20250510,negative,NaN,1,1_24005_1,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000
1,1,24005,307482,2025-05-10 13:42:39.811,NaN,20250510,negative,NaN,2,1_24005_2,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000
2,1,252326,169041,2025-05-09 23:05:42.101,NaN,20250509,negative,NaN,1,1_252326_1,1_252326,24-30,F,893,50000-100000,18-23,F,Other,178,100-500,10000-100000
3,1,264714,193781,2025-05-11 00:37:06.659,NaN,20250511,negative,NaN,1,1_264714_1,1_264714,24-30,F,893,50000-100000,18-23,M,Chat,90,100-500,100000-1000000
4,1,266768,430309,2025-05-05 12:25:33.964,NaN,20250505,negative,NaN,1,1_266768_1,1_266768,24-30,F,893,50000-100000,18-23,F,Talent,688,1000-5000,1000000-2000000


In [91]:
interactions.shape

(18003780, 21)

In [92]:
# interactions - map with room and take live_dur, live_content_category
interactions = interactions.merge(room_cl[['live_id', 'live_dur', 'live_content_category']], on='live_id', how='left')
interactions.rename(columns={'live_dur': 'room_live_dur', 'live_content_category': 'room_live_content_category'}, inplace=True) 
interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,p_date,interaction_type,gift_price,interaction_order,pk,session_id,user_age,user_gender,user_active_time,user_accu_watch_live_cnt,streamer_age,streamer_gender,streamer_live_operation_tag,streamer_active_time,streamer_accu_live_cnt,streamer_fans_user_num,room_live_dur,room_live_content_category
0,1,24005,307482,2025-05-10 12:46:21.212,NaN,20250510,negative,NaN,1,1_24005_1,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000,236.369050,other
1,1,24005,307482,2025-05-10 13:42:39.811,NaN,20250510,negative,NaN,2,1_24005_2,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000,236.369050,other
2,1,252326,169041,2025-05-09 23:05:42.101,NaN,20250509,negative,NaN,1,1_252326_1,1_252326,24-30,F,893,50000-100000,18-23,F,Other,178,100-500,10000-100000,38.626033,other
3,1,264714,193781,2025-05-11 00:37:06.659,NaN,20250511,negative,NaN,1,1_264714_1,1_264714,24-30,F,893,50000-100000,18-23,M,Chat,90,100-500,100000-1000000,93.657467,other
4,1,266768,430309,2025-05-05 12:25:33.964,NaN,20250505,negative,NaN,1,1_266768_1,1_266768,24-30,F,893,50000-100000,18-23,F,Talent,688,1000-5000,1000000-2000000,82.352650,other


In [93]:
interactions.shape

(18003780, 23)

In [96]:
room_cl.head()

,p_date,live_id,streamer_id,live_type,start_timestamp,end_timestamp,live_content_category,live_dur,live_dur_day
89,20250505,4207342,439076,1,2025-05-04 23:08:35.691,2025-05-05 00:00:00.035,other,51.405733,0 days 00:51:24.344000
90,20250505,6894755,356465,1,2025-05-04 22:35:55.074,2025-05-05 00:00:00.051,other,84.082950,0 days 01:24:04.977000
91,20250505,8396644,411360,1,2025-05-04 23:42:30.419,2025-05-05 00:00:00.093,other,17.494567,0 days 00:17:29.674000
92,20250505,4473195,271581,1,2025-05-04 22:35:47.484,2025-05-05 00:00:00.157,other,84.211217,0 days 01:24:12.673000
93,20250505,6447876,446221,1,2025-05-04 23:26:37.516,2025-05-05 00:00:00.253,other,33.378950,0 days 00:33:22.737000


# 2. Export to csv files

In [99]:
# now export to csv files for each dataframe to folder clean file
def export_to_csv(df, filename):
    """
    Export the dataframe to a CSV file.
    """
    df.to_csv(f'cleaned_dataset/{filename}', index=False)

# export all dataframes to csv files
export_to_csv(click_cl, 'click_cl.csv')
export_to_csv(comment_cl, 'comment_cl.csv')
export_to_csv(gift_cl, 'gift_cl.csv')
export_to_csv(like_cl, 'like_cl.csv')
export_to_csv(negative_cl, 'negative_cl.csv')
export_to_csv(room_cl, 'room_cl.csv')
export_to_csv(streamer_cl, 'streamer_cl.csv')
export_to_csv(user_cl, 'user_cl.csv')
export_to_csv(interactions, 'interactions.csv')

In [101]:
negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12705835 entries, 0 to 12705834
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   live_id      int64         
 2   streamer_id  int64         
 3   timestamp    datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 387.8 MB


In [102]:
interactions.head()

,user_id,live_id,streamer_id,timestamp,watch_live_time,p_date,interaction_type,gift_price,interaction_order,pk,session_id,user_age,user_gender,user_active_time,user_accu_watch_live_cnt,streamer_age,streamer_gender,streamer_live_operation_tag,streamer_active_time,streamer_accu_live_cnt,streamer_fans_user_num,room_live_dur,room_live_content_category
0,1,24005,307482,2025-05-10 12:46:21.212,NaN,20250510,negative,NaN,1,1_24005_1,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000,236.369050,other
1,1,24005,307482,2025-05-10 13:42:39.811,NaN,20250510,negative,NaN,2,1_24005_2,1_24005,24-30,F,893,50000-100000,24-30,M,Chat,2811,1000-5000,3000000-4000000,236.369050,other
2,1,252326,169041,2025-05-09 23:05:42.101,NaN,20250509,negative,NaN,1,1_252326_1,1_252326,24-30,F,893,50000-100000,18-23,F,Other,178,100-500,10000-100000,38.626033,other
3,1,264714,193781,2025-05-11 00:37:06.659,NaN,20250511,negative,NaN,1,1_264714_1,1_264714,24-30,F,893,50000-100000,18-23,M,Chat,90,100-500,100000-1000000,93.657467,other
4,1,266768,430309,2025-05-05 12:25:33.964,NaN,20250505,negative,NaN,1,1_266768_1,1_266768,24-30,F,893,50000-100000,18-23,F,Talent,688,1000-5000,1000000-2000000,82.352650,other


In [103]:
interactions.shape

(18003780, 23)